In [1]:
import os 
import pandas as pd
import pickle
import torch
import wandb

os.environ["WANDB_QUIET"] = "true"
os.environ["WANDB_NOTEBOOK_NAME"] = "dynamic_decision_opt.ipynb"
project_name = "Dynamic Inventory Management for Bimbo"
decision_data = pd.read_parquet('./data/bimbo/decision_data.parquet')
decision_data.head()

,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Semana,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
5030470,1110,7,3301,15766,1238,5,1,9.83,0,0.0,1
5030471,1110,7,3301,15766,1242,5,2,15.28,0,0.0,2
5030472,1110,7,3301,15766,1250,5,8,61.12,0,0.0,8
5030473,1110,7,3301,15766,1309,5,6,40.56,0,0.0,6
5030474,1110,7,3301,15766,3894,5,8,74.80,0,0.0,8


In [2]:
sample_store_and_product = decision_data.query(
"Agencia_ID == 1110 & Canal_ID == 7 & Ruta_SAK == 3301 & Cliente_ID == 15766 & Producto_ID == 1238"
)

In [3]:
store_product_ids = [
    "Agencia_ID",
    "Canal_ID",
    "Ruta_SAK",
    "Cliente_ID",
    "Producto_ID",
]

numerical_cols = [
    "Venta_uni_hoy",
    "Venta_hoy",
]

model = torch.load("predictive_model.pt")
with open('catgeorical_encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

categorical_for_prediction = sample_store_and_product[store_product_ids].values
categorical_encoded = encoder.transform(categorical_for_prediction)
categorical_tensor = torch.from_numpy(categorical_encoded).long()
categorical_tensor = [categorical_tensor[:, i] for i in range(categorical_tensor.shape[1])]

numerical_tensor = torch.from_numpy(sample_store_and_product[numerical_cols].values).float()
model.eval()
with torch.no_grad():
    prediction = model(categorical_tensor, numerical_tensor)
prediction

tensor([1.0019, 1.8646, 1.8646, 2.5598, 1.8646])

In [4]:
sample_store_and_product = sample_store_and_product.assign(predicted_demand = prediction.numpy())
sample_store_and_product

,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Semana,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,predicted_demand
5030470,1110,7,3301,15766,1238,5,1,9.83,0,0.0,1,1.001867
7545705,1110,7,3301,15766,1238,6,2,19.66,0,0.0,2,1.864592
10060940,1110,7,3301,15766,1238,7,2,19.66,0,0.0,2,1.864592
12576175,1110,7,3301,15766,1238,8,3,29.49,0,0.0,3,2.559850
15091410,1110,7,3301,15766,1238,9,2,19.66,0,0.0,2,1.864592


In [5]:
def add_preds_to_df(df):
    categorical_for_prediction = df[store_product_ids].values
    categorical_encoded = encoder.transform(categorical_for_prediction)
    categorical_tensor = torch.from_numpy(categorical_encoded).long()
    categorical_tensor = [categorical_tensor[:, i] for i in range(categorical_tensor.shape[1])]
    numerical_tensor = torch.from_numpy(df[numerical_cols].values).float()
    model.eval()
    with torch.no_grad():
        prediction = model(categorical_tensor, numerical_tensor)
    return df.assign(predicted_demand = prediction.numpy())

sample_store_and_product = decision_data.query(
"Agencia_ID == 1110 & Canal_ID == 7 & Ruta_SAK == 3301 & Cliente_ID == 15766 & Producto_ID == 1238"
)
add_preds_to_df(sample_store_and_product)

,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Semana,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,predicted_demand
5030470,1110,7,3301,15766,1238,5,1,9.83,0,0.0,1,1.001867
7545705,1110,7,3301,15766,1238,6,2,19.66,0,0.0,2,1.864592
10060940,1110,7,3301,15766,1238,7,2,19.66,0,0.0,2,1.864592
12576175,1110,7,3301,15766,1238,8,3,29.49,0,0.0,3,2.559850
15091410,1110,7,3301,15766,1238,9,2,19.66,0,0.0,2,1.864592


In [6]:
def add_col_with_initial_value(df, col_name, value):
    df.loc[df.index[0], col_name] = value
    return df

def simulate_outcomes(df, decision_rule):
    df = df.copy()
    df = add_preds_to_df(df)
    df = add_col_with_initial_value(df, "old_stock", 0)
    first_stocking_decision = decision_rule(df.iloc[0])
    df = add_col_with_initial_value(df, "new_stock", first_stocking_decision)
    first_shortage = max(0, df.iloc[0].predicted_demand - df.iloc[0].new_stock)
    first_amount_sold = min(df.iloc[0].Demanda_uni_equil, df.iloc[0].new_stock + df.iloc[0].old_stock)
    df = add_col_with_initial_value(df, "shortage", first_shortage)
    df = add_col_with_initial_value(df, "total_sold", first_amount_sold)

    # Sometimes can use .shift pattern
    prev_period = df.iloc[0, :]
    for i in df.index[1:]:
        df.loc[i, "old_stock"] = max(0,
                                    min(prev_period.old_stock + prev_period.new_stock - prev_period.Demanda_uni_equil,
                                     prev_period.new_stock
                                    ))
        df.loc[i, "new_stock"] = decision_rule(df.loc[i])
        stock_on_hand = df.loc[i, "old_stock"] + df.loc[i, "new_stock"]
        df.loc[i, "shortage"] = max(0, df.loc[i, "Demanda_uni_equil"] - stock_on_hand)
        df.loc[i, "total_sold"] = min(df.loc[i, "Demanda_uni_equil"], stock_on_hand)
        df.loc[i, "spoilage"] = max(0, df.loc[i, "old_stock"] - df.loc[i, "Demanda_uni_equil"])
        prev_period = df.loc[i]
    return df

def first_decision_rule(state):
    return 1

simulate_outcomes(sample_store_and_product, first_decision_rule)

,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Semana,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,predicted_demand,old_stock,new_stock,shortage,total_sold,spoilage
5030470,1110,7,3301,15766,1238,5,1,9.83,0,0.0,1,1.001867,0.0,1.0,0.001867,1.0,NaN
7545705,1110,7,3301,15766,1238,6,2,19.66,0,0.0,2,1.864592,0.0,1.0,1.000000,1.0,0.0
10060940,1110,7,3301,15766,1238,7,2,19.66,0,0.0,2,1.864592,0.0,1.0,1.000000,1.0,0.0
12576175,1110,7,3301,15766,1238,8,3,29.49,0,0.0,3,2.559850,0.0,1.0,2.000000,1.0,0.0
15091410,1110,7,3301,15766,1238,9,2,19.66,0,0.0,2,1.864592,0.0,1.0,1.000000,1.0,0.0


## Scaling to multiple stores

In [7]:
decision_data.groupby('Agencia_ID').size()

Agencia_ID
1110      14270
1111     108890
1112      74115
1113      43345
1114      23035
          ...  
24049     14915
24539     16765
24669     23115
25699     12140
25759      1190
Length: 545, dtype: int64

In [8]:
decision_validation_data = decision_data.query('Agencia_ID == 1110')
decision_holdout_data = decision_data.query('Agencia_ID == 24049')

In [9]:
def objective_function(df):
    return df.total_sold.sum() - 3*df.shortage.sum() - 0.5 * df.spoilage.sum() - 0.5*df.old_stock.sum()

def log_metrics(outcomes, decision_function, tags=None):
    with wandb.init(project=project_name,
                    name=decision_function.__name__,
                    job_type="simulation outcomes",
                    tags=tags
                    ):
        wandb.log({
            "number_of_orders": outcomes.new_stock.count(),
            "total_inventory_orders": outcomes.new_stock.sum(),
            "number_of_shortages": (outcomes.shortage > 0).sum(),
            "total_shortage": outcomes.shortage.sum(),
            "total_sold": outcomes.total_sold.sum(),
            "total_old_stock": outcomes.old_stock.sum(),
            "full_outcome": outcomes[store_product_ids + ['Semana', 'old_stock', 'new_stock', 'shortage', 'total_sold', 'spoilage']],
            "objective_function": objective_function(outcomes)
        })
    return

def simulate_multiple_stores_and_products(raw_data, decision_function, tags, log=True):
    groups = raw_data.groupby(store_product_ids)
    outcomes = pd.concat([simulate_outcomes(group, decision_function) for _, group in groups])
    if log:
        log_metrics(outcomes, decision_function, tags)
    return outcomes

simulate_multiple_stores_and_products(decision_validation_data, first_decision_rule, tags=["agencia_1110"])

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find dynamic_decision_opt.ipynb.


,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Semana,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,predicted_demand,old_stock,new_stock,shortage,total_sold,spoilage
5030470,1110,7,3301,15766,1238,5,1,9.83,0,0.0,1,1.001867,0.0,1.0,0.001867,1.0,NaN
7545705,1110,7,3301,15766,1238,6,2,19.66,0,0.0,2,1.864592,0.0,1.0,1.000000,1.0,0.0
10060940,1110,7,3301,15766,1238,7,2,19.66,0,0.0,2,1.864592,0.0,1.0,1.000000,1.0,0.0
12576175,1110,7,3301,15766,1238,8,3,29.49,0,0.0,3,2.559850,0.0,1.0,2.000000,1.0,0.0
15091410,1110,7,3301,15766,1238,9,2,19.66,0,0.0,2,1.864592,0.0,1.0,1.000000,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5033323,1110,11,3504,4706818,1146,5,1,21.39,0,0.0,1,1.042666,0.0,1.0,0.042666,1.0,NaN
7548558,1110,11,3504,4706818,1146,6,2,42.78,0,0.0,2,1.914276,0.0,1.0,1.000000,1.0,0.0
10063793,1110,11,3504,4706818,1146,7,1,21.39,0,0.0,1,1.042666,0.0,1.0,0.000000,1.0,0.0
12579028,1110,11,3504,4706818,1146,8,2,42.78,0,0.0,2,1.914276,0.0,1.0,1.000000,1.0,0.0


In [10]:
import numpy as np

def predicted_need(state):
    return np.ceil(state.predicted_demand - state.old_stock)

def predicted_need_plus_one(state):
    return predicted_need(state) + 1

def predicted_demand(state):
    return np.ceil(state.predicted_demand)

for rule in [first_decision_rule, predicted_need, predicted_need_plus_one, predicted_demand]:
    simulate_multiple_stores_and_products(decision_validation_data, rule, tags=["agencia_1110"])


# Programmatic Optimization

In [11]:
def linear_decision_function_factory(constant, predicted_demand_mult, old_stock_mult):
    def decision_function(state):
        return constant + predicted_demand_mult * state.predicted_demand + old_stock_mult * state.old_stock
    return decision_function

def objective(params):
    decision_function = linear_decision_function_factory(params.constant, params.predicted_demand_mult, params.old_stock_mult)
    outcomes = simulate_multiple_stores_and_products(decision_validation_data, decision_function, tags=["agencia_1110"], log=False)
    return objective_function(outcomes)

sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'objective_function',
        'goal': 'maximize'
    },
    'parameters': {
        'constant': {
            'distribution': 'uniform',
            'min': 0,
            'max': 5
        },
        'predicted_demand_mult': {
            'distribution': 'uniform',
            'min': 0,
            'max': 1.5
        },
        'old_stock_mult': {
            'distribution': 'uniform',
            'min': -1.5,
            'max': 0,
        }
    }
}

def main():
    wandb.init(project=project_name)
    score = objective(wandb.config)
    wandb.log({'objective_function': score})

sweep_id = wandb.sweep(sweep_config, project=project_name)
wandb.agent(sweep_id, main, count=20)

Create sweep with ID: lbluda5b
Sweep URL: https://wandb.ai/danbecker/Dynamic%20Inventory%20Management%20for%20Bimbo/sweeps/lbluda5b


wandb: Agent Starting Run: f5tvhb9q with config:
wandb: 	constant: 1.695053619214561
wandb: 	old_stock_mult: -1.0480853784746338
wandb: 	predicted_demand_mult: 0.19530161088577075
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find dynamic_decision_opt.ipynb.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: yh3o3cwc with config:
wandb: 	constant: 2.87896143271543
wandb: 	old_stock_mult: -0.005086160307670173
wandb: 	predicted_demand_mult: 1.392191620002041
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find dynamic_decision_opt.ipynb.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: Agent Starting Run: 0byx6nvn with config:
wandb: 	constant: 4.005385495815188
wandb: 	old_stock_mult: -0.43922425520563824
wandb: 	predicted_demand_mult: 1.4724187618891382
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find dynamic_decision_opt.ipynb.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: Agent Starting Run: bctc2jg1 with config:
wandb: 	constant: 4.7782854123306855
wandb: 	old_stock_mult: -0.14348988858048295
wandb: 	predicted_demand_mult: 1.4983451976149582
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find dynamic_decision_opt.ipynb.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

wandb: Agent Starting Run: tif8dcbl with config:
wandb: 	constant: 0.04985098585556125
wandb: 	old_stock_mult: -1.2936424954733878
wandb: 	predicted_demand_mult: 1.4974187866444155
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find dynamic_decision_opt.ipynb.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: Agent Starting Run: od8kt876 with config:
wandb: 	constant: 4.942795004512041
wandb: 	old_stock_mult: -1.4158397501463005
wandb: 	predicted_demand_mult: 1.4026971639959367
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find dynamic_decision_opt.ipynb.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: Agent Starting Run: 5h6hcvyu with config:
wandb: 	constant: 0.06236260187204412
wandb: 	old_stock_mult: -1.0360744257323558
wandb: 	predicted_demand_mult: 1.4784998920641117
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find dynamic_decision_opt.ipynb.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: Agent Starting Run: haoakdfc with config:
wandb: 	constant: 4.94249705406472
wandb: 	old_stock_mult: -1.47909320002831
wandb: 	predicted_demand_mult: 1.4358260656936217
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find dynamic_decision_opt.ipynb.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: Agent Starting Run: fde8sdow with config:
wandb: 	constant: 1.90757922910494
wandb: 	old_stock_mult: -1.4918831271107054
wandb: 	predicted_demand_mult: 1.47856134668134
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find dynamic_decision_opt.ipynb.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
